In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./CombinedObjectiveFile_Products_080122.csv')
num_rows = df.shape[0]
df.insert(6, 'Net_Carbon', [0] * num_rows)

START_YEAR = 2021
FOR_TYPES = list(set(df['StandID']))
YEARS = list(set(df['Year']))
MNGS = list(set(df['MgmtID']))

df.sort_values(['StandID', 'Year'])

,Variable,StandID,MgmtID,Year,Total_Stand_Carbon,HrvCarbon_Products,Net_Carbon
0,167N_2021_PLSQ,167N,PLSQ,2021,35.021587,0.000000,0
1,167N_2021_THNB,167N,THNB,2021,34.437302,0.290956,0
2,167N_2025_PLSQ,167N,PLSQ,2025,36.869949,0.000000,0
3,167N_2025_THNB,167N,THNB,2025,35.482277,0.191104,0
4,167N_2030_PLSQ,167N,PLSQ,2030,36.483826,0.000000,0
5,167N_2030_THNB,167N,THNB,2030,36.572166,0.123852,0
6,167N_2050_PLSQ,167N,PLSQ,2050,43.433018,0.000000,0
7,167N_2050_THNB,167N,THNB,2050,57.759636,0.067200,0
8,167S_2021_PLSQ,167S,PLSQ,2021,37.369804,0.000000,0
9,167S_2021_THNB,167S,THNB,2021,36.438778,0.436180,0


## What We Want To Do
 - 1) Turn stand carbon into net sequestration
 - 2) Add harvested product
 - 3) Subtract carbon costs

## 1 - Net Sequestration
We add a new column, net_carbon, which for each entry is equal to the net change in Total_Stand_Carbon from
one year to the next.

For 2021, we compare against the no management (PLSQ) for each forest type.

In [3]:
# Baseline extraction - we generate a dataframe which gives the Start Year PLSQ for each forest type
df_baseline = df[df['Year'] == START_YEAR]
df_baseline = df_baseline[df_baseline['MgmtID'] == 'PLSQ']

# We only care about these two columns
df_baseline = df_baseline[['Year', 'StandID', 'Total_Stand_Carbon']]
df_baseline = df_baseline.rename(columns={
    'Total_Stand_Carbon': 'Baseline_Carbon_2021'
})
df_baseline

,Year,StandID,Baseline_Carbon_2021
0,2021,167N,35.021587
8,2021,167S,37.369804
18,2021,505,55.080078
33,2021,608,45.944687
43,2021,999,7.028458


In [4]:
# Calculate Net Carbon
# Step 1: Take difference between year to year

# This is achieved by sorting correctly, and then subtracting consecutive rows
# See: https://datagy.io/pandas-shift/
df_seq = df.sort_values(['StandID', 'MgmtID'])
df_seq['Shifted Carbon'] = df_seq['Total_Stand_Carbon'].shift(periods=1, fill_value=0)
df_seq['Net_Carbon'] = df_seq['Total_Stand_Carbon'] - df_seq['Shifted Carbon']
# df_seq

In [5]:
# Calculate Net Carbon
# Step 2: For 2021, subtract against PLSQ

# This adds a column with the 2021 PLSQ baseline numbers
df_seq = df_seq.merge(df_baseline, 
         how='left', 
         on=['Year', 'StandID'], 
         validate='many_to_one'
        )

df_seq['First_Year_Net_Carbon'] = df_seq['Total_Stand_Carbon'] - df_seq['Baseline_Carbon_2021']
df_seq['Net_Carbon'] = df_seq['First_Year_Net_Carbon'].fillna(df_seq['Net_Carbon'])
# df_seq

In [6]:
# Cleanup - remove now unnecessary rows
df_seq = df_seq.drop(columns=[
    'Shifted Carbon', 
    'Baseline_Carbon_2021', 
    'First_Year_Net_Carbon',
    'Total_Stand_Carbon'
])

df_seq

,Variable,StandID,MgmtID,Year,HrvCarbon_Products,Net_Carbon
0,167N_2021_PLSQ,167N,PLSQ,2021,0.000000,0.000000
1,167N_2025_PLSQ,167N,PLSQ,2025,0.000000,1.848362
2,167N_2030_PLSQ,167N,PLSQ,2030,0.000000,-0.386124
3,167N_2050_PLSQ,167N,PLSQ,2050,0.000000,6.949192
4,167N_2021_THNB,167N,THNB,2021,0.290956,-0.584286
5,167N_2025_THNB,167N,THNB,2025,0.191104,1.044975
6,167N_2030_THNB,167N,THNB,2030,0.123852,1.089890
7,167N_2050_THNB,167N,THNB,2050,0.067200,21.187469
8,167S_2021_PLSQ,167S,PLSQ,2021,0.000000,0.000000
9,167S_2025_PLSQ,167S,PLSQ,2025,0.000000,5.697899


## 2 - Add Carbon Harvest
The data has a column of carbon product harvested. This adds that to the Net_Carbon

In [7]:
df_hrv = df_seq
# df_hrv['Net_Carbon'] = df_seq['Net_Carbon'] + df_seq['HrvCarbon_Products']

# Now we can drop the HrvCarbonProduct Table
df_hrv = df_hrv.drop(columns=['HrvCarbon_Products'])
df_hrv

,Variable,StandID,MgmtID,Year,Net_Carbon
0,167N_2021_PLSQ,167N,PLSQ,2021,0.000000
1,167N_2025_PLSQ,167N,PLSQ,2025,1.848362
2,167N_2030_PLSQ,167N,PLSQ,2030,-0.386124
3,167N_2050_PLSQ,167N,PLSQ,2050,6.949192
4,167N_2021_THNB,167N,THNB,2021,-0.584286
5,167N_2025_THNB,167N,THNB,2025,1.044975
6,167N_2030_THNB,167N,THNB,2030,1.089890
7,167N_2050_THNB,167N,THNB,2050,21.187469
8,167S_2021_PLSQ,167S,PLSQ,2021,0.000000
9,167S_2025_PLSQ,167S,PLSQ,2025,5.697899


## 3 - Add Carbon Costs
Here, we take a user specified cost of each mgmt by forest type. 

We probably also need to scale it up by years since not all periods are equal length but that's future work -\\\_(ツ)_/-

In [9]:
COSTS_DICT = {
    '167N': {
        'PLSQ': 0,
        'THNB': 10
    },
    '167S': {
        'PLSQ': 0,
        'THNB': 10
    },
    '505': {
        'PLSQ': 0,
        'ASV': 0,
        'IFM': 0,
        'THNB': 10
    },
    '608': {
        'PLSQ': 0,
        'AWR': 0
    },
    '999': {
        'PLSQ': 0,
        'AWR': 0,
        'CAR': 0,
        'NAR': 0,
        'SAR': 0
    }
}

In [20]:
# The format of the dict isn't actually all that good for converting to a dataframe
# so we need some ~finangeling~ for it to work
datadict = {
    'StandID': [],
    'MgmtID': [],
    'Mgmt_Cost': []
}

for_types = COSTS_DICT.keys()
for f in for_types:
    mgmts = COSTS_DICT[f].keys()
    for mg in mgmts:
        cost = COSTS_DICT[f][mg]
        
        datadict['StandID'].append(f)
        datadict['MgmtID'].append(mg)
        datadict['Mgmt_Cost'].append(cost)

df_costs = pd.DataFrame.from_dict(datadict)
df_costs

,StandID,MgmtID,Mgmt_Cost
0,167N,PLSQ,0
1,167N,THNB,10
2,167S,PLSQ,0
3,167S,THNB,10
4,505,PLSQ,0
5,505,ASV,0
6,505,IFM,0
7,505,THNB,10
8,608,PLSQ,0
9,608,AWR,0


In [28]:
# Now we append the management cost to the main df, and subtract it from net_carbon
df_mgcost = df_hrv.merge(
    df_costs,
    how='left',
    on=['StandID', 'MgmtID'],
    validate='many_to_one'
    )

df_mgcost['Net_Carbon'] = df_mgcost['Net_Carbon'] - df_mgcost['Mgmt_Cost']

In [27]:
df_mgcost

,Variable,StandID,MgmtID,Year,Net_Carbon,Mgmt_Cost
0,167N_2021_PLSQ,167N,PLSQ,2021,0.000000,0
1,167N_2025_PLSQ,167N,PLSQ,2025,1.848362,0
2,167N_2030_PLSQ,167N,PLSQ,2030,-0.386124,0
3,167N_2050_PLSQ,167N,PLSQ,2050,6.949192,0
4,167N_2021_THNB,167N,THNB,2021,-10.584286,10
5,167N_2025_THNB,167N,THNB,2025,-8.955025,10
6,167N_2030_THNB,167N,THNB,2030,-8.910110,10
7,167N_2050_THNB,167N,THNB,2050,11.187469,10
8,167S_2021_PLSQ,167S,PLSQ,2021,0.000000,0
9,167S_2025_PLSQ,167S,PLSQ,2025,5.697899,0


## Exporting

In [30]:
df_mgcost.to_csv('./ObjectiveFile_MngCost_0802022.csv',
              columns=['Variable', 'Net_Carbon'],
              index=False
             )